In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Data Prepration

In [7]:
from datasets import load_dataset, Dataset
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
dataset[0]

{'text': '<human>: What are some tips for creating a successful business plan?\n<bot>: \n1. Research your target market: Understand who you are selling to, how much competition there is, and the current trends.\n2. Determine your business model and strategies: Decide what products and services you will offer, and how you will compete in the market.\n3. Outline your financial projections: Estimate start-up costs, revenue projections, and operating costs.\n4. Analyze the risks: Identify potential problems and create strategies to mitigate them.\n5. Develop a marketing plan: Strategize how you will promote your business and reach new customers.\n6. Set milestones: Establish short-term and long-term goals and devise a plan to measure progress.',
 'metadata': {'source': 'unified_chip2'}}

In [11]:
data_df = dataset.to_pandas()
data_df

,text,metadata
0,<human>: What are some tips for creating a suc...,{'source': 'unified_chip2'}
1,<human>: Describe the location where you find ...,{'source': 'unified_chip2'}
2,<human>: I am interested in gaining an underst...,{'source': 'unified_chip2'}
3,<human>: I want to find out more about a certa...,{'source': 'unified_chip2'}
4,<human>: round number to nearest integer\n<bot...,{'source': 'unified_chip2'}
...,...,...
210284,<human>: What are some of the most popular nat...,{'source': 'unified_chip2'}
210285,<human>: When I type a command in the Blit Ter...,{'source': 'unified_chip2'}
210286,"<human>: I want to start a business, what are ...",{'source': 'unified_chip2'}
210287,<human>: I am looking for free online courses ...,{'source': 'unified_chip2'}


In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args=args,

)

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 210,289 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,2.857200
2,2.296400
3,2.684900
4,2.248600
5,2.812100
6,2.423200
7,2.502300
8,1.705800
9,1.498200
10,1.376300


Unsloth: Will smartly offload gradients to save VRAM!


In [17]:
model.save_pretrained("lora_model")
model.save_pretrained_merged("outputs", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("Risha6h/Llama-3.2-3B-Instructunsloth-merged", tokenizer, save_method = "merged_16bit", token = os.environ.get("HF_TOKEN"))

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.95 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 26.70it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving outputs/pytorch_model-00001-of-00002.bin...
Unsloth: Saving outputs/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: You are pushing to hub, but you passed your HF username = Risha6h.
We shall truncate Risha6h/Llama-3.2-3B-Instructunsloth-merged to Llama-3.2-3B-Instructunsloth-merged


...

README.md:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/Risha6h/Llama-3.2-3B-Instructunsloth-merged


In [21]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=150)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

generate_text("What are tips for successful business plan?")

What are tips for successful business plan??
Here are some tips for creating a successful business plan:
1. **Define your mission and vision**: Clearly outline your business goals, values, and objectives.
2. **Conduct market research**: Understand your target audience, industry trends, and competitors.
3. **Develop a unique value proposition**: Explain how your business will differentiate itself from others.
4. **Create a detailed financial plan**: Outline projected income, expenses, and funding requirements.
5. **Establish a marketing strategy**: Outline how you will reach and engage with your target audience.
6. **Develop a sales strategy**: Outline how you will generate revenue and meet customer needs.
7. **Create a management plan**: Outline how you will manage and lead your team.
8. **Establish


In [22]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
8.002 GB of memory reserved.


In [26]:
import json
import requests

# Dataset URL
dataset_url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"

# Fetch data from the URL
response = requests.get(dataset_url)
response.raise_for_status()  # Ensure a valid response

# Convert JSONL to a list of dictionaries
raw_data = [json.loads(line) for line in response.text.splitlines()]

# Function to extract input and response
def extract_prompt_response(text):
    parts = text.split("<bot>:")
    if len(parts) < 2:
        return None, None  # Skip if format is incorrect
    prompt = parts[0].replace("<human>:", "").strip()
    response = parts[1].strip()
    return prompt, response

# Process the dataset
test_prompts, expected_outputs = [], []
for item in raw_data:
    prompt, response = extract_prompt_response(item["text"])
    if prompt and response:  # Ensure valid data
        test_prompts.append(prompt)
        expected_outputs.append(response)

# Sample Output
print(f"Loaded {len(test_prompts)} valid prompt-response pairs.")
print("Example Prompt:", test_prompts[0])
print("Expected Output:", expected_outputs[0])


Loaded 210289 valid prompt-response pairs.
Example Prompt: What are some tips for creating a successful business plan?
Expected Output: 1. Research your target market: Understand who you are selling to, how much competition there is, and the current trends.
2. Determine your business model and strategies: Decide what products and services you will offer, and how you will compete in the market.
3. Outline your financial projections: Estimate start-up costs, revenue projections, and operating costs.
4. Analyze the risks: Identify potential problems and create strategies to mitigate them.
5. Develop a marketing plan: Strategize how you will promote your business and reach new customers.
6. Set milestones: Establish short-term and long-term goals and devise a plan to measure progress.
